In [ ]:
import set_path

import numpy as np
import laspy
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
import pandas as pd

from upcp.utils import ahn_utils, clip_utils
from upcp.utils.interpolation import FastGridInterpolator

from gvl.helper_functions import color_clusters
from gvl.tree_detector import DetectorTree
from gvl.ahn_utils import GeoTIFFReader2

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
RD_CRS = 'epsg:28992'
LL_CRS = 'WGS84'

# AHN classification
AHN_OTHER = 1
AHN_GROUND = 2
AHN_BUILDING = 6
AHN_WATER = 9
AHN_ARTIFACT = 26

In [ ]:
# # Use CycloMedia style tilecodes
# tilecode = '2463_9714'

# las_file = f'../datasets/ahn_{tilecode}.laz'
# ahn_npz_folder = '../../../datasets/Accessibility_oost/ahn/ahn4_npz/'

# ahn_reader = ahn_utils.NPZReader(ahn_npz_folder, caching=False)

### TODO make geotiles scraper
We use 5m resolution. The ground filter improvements when using is 0.5m is negligible and running time increases.

In [ ]:
# Use AHN subtiles
tilecode = '25GN1_04'

las_file = f'../datasets/ahn_laz/{tilecode}.LAZ'
ahn_geotiff_folder = '../datasets/ahn_dtm/'

ahn_reader = GeoTIFFReader2(ahn_geotiff_folder, fill_gaps=False,
                            smoothen=True, smooth_thickness=2)

In [ ]:
# target_area = gpd.read_file(f'../datasets/validation/correct_trees_areas_oosterpark.shp', crs=RD_CRS)
# one_poly = target_area['geometry'].iloc[1]

In [ ]:
target_area_points = gpd.read_file(f'../datasets/validation/joined_trees_bgt_gissib_1_5_amsterdam.shp', crs=RD_CRS)
#bbox = target_area.unary_union.bounds
#bbox = one_poly.bounds

In [ ]:
target_area_points = target_area_points.to_crs(RD_CRS)

In [ ]:
las = laspy.read(las_file)
pts = np.vstack((las.x, las.y, las.z)).T
pts_rgb = np.vstack((las.red, las.green, las.blue)).T

In [ ]:
pc_header = las.header
offset = 50 # Necesarry to avoid incorrect DTM + LAZ overlap
bbox_original = pc_header.min[0], pc_header.min[1], pc_header.max[0], pc_header.max[1]
bbox = pc_header.min[0]+offset, pc_header.min[1]+offset, pc_header.max[0]-offset, pc_header.max[1]-offset

### Filter 1: scalar fields and clip
- classification -> 'overig label' points

We dont want to filter in the number_of_returns scalar field. It will remove too much valuable tree points. 

In [ ]:
area_mask = clip_utils.rectangle_clip(pts, bbox)
class_mask = las.classification == AHN_OTHER

mask = class_mask & area_mask

In [ ]:
gnd_tile = ahn_reader.filter_area(bbox)

fast_z = FastGridInterpolator(
            gnd_tile['x'], gnd_tile['y'], gnd_tile['ground_surface'])
gnd_z = fast_z(pts[mask])

In [ ]:
np.count_nonzero(gnd_z == np.nan)

### Filter 2: points close to ground
TODO move this before the KDTree code

In [ ]:
above_ground_in_meters = 2.5
height_mask = pts[mask,2] - gnd_z >= above_ground_in_meters

In [ ]:
raw_points = pts[mask][height_mask]
raw_points_rgb = pts_rgb[mask][height_mask]

In [ ]:
# Ugggly
ground_z = gnd_z[height_mask]

In [ ]:
ja = list(zip(*raw_points))
gek = list(zip(*raw_points_rgb))

points_3d = np.vstack((ja[0], ja[1], ja[2])).T
points_rgb = np.vstack((gek[0], gek[1], gek[2])).T

# LCC

In [ ]:
from upcp.utils.clip_utils import poly_box_clip
from upcp.utils.math_utils import minimum_bounding_rectangle

def _label_tree_like_components(points, ground_z, point_components,
                               road_polygons, min_height):
    """ Based on certain properties of a tree we label clusters.  """
    
    tree_mask = np.zeros(len(points), dtype=bool)
    tree_count = 0

    cc_labels = np.unique(point_components)

    cc_labels = set(cc_labels).difference((-1,))

    for cc in cc_labels:
        # select points that belong to the cluster
        cc_mask = (point_components == cc)

        target_z = ground_z[cc_mask]
        valid_values = target_z[np.isfinite(target_z)]

        if valid_values.size != 0:
            cc_z = np.mean(valid_values)
            min_z = cc_z + min_height
            cluster_height = np.amax(points[cc_mask][:, 2])
            if min_z <= cluster_height:
                mbrect, conv_hull, mbr_width, mbr_length, _ =\
                    minimum_bounding_rectangle(points[cc_mask][:, :2])
                p1 = Polygon(conv_hull)
                for p2 in road_polygons:
                    do_overlap = p1.intersects(p2)
                    if do_overlap:
                        tree_mask[cc_mask] = True
                        tree_count += 1
                        break

    print(f'{tree_count} tree labelled.')

    return tree_mask

In [ ]:
def bbox_to_polygon(bbox):
    polygon = box(*bbox, ccw=True)
    return polygon

bbox_shapely = bbox_to_polygon(bbox_original)

In [ ]:
bbox_df = gpd.GeoDataFrame(pd.DataFrame(['p1'], columns = ['geom']),
         crs = {'init':'epsg:28992'},
         geometry = [bbox_shapely])

In [ ]:
within_points = gpd.sjoin(target_area_points, bbox_df, op = 'within')

In [ ]:
tree_points = list(within_points['geometry'].values) # gp_df

In [ ]:
from upcp.region_growing.label_connected_comp import LabelConnectedComp

label_mask = np.zeros((len(points_3d),), dtype=bool)

if len(tree_points) == 0:
    print('No reference tree points, skipping.')

grid_size = 0.6
min_component_size = 50
min_height = 3.5

lcc = LabelConnectedComp(grid_size=grid_size,
                         min_component_size=min_component_size)
point_components = lcc.get_components(points_3d)

# Label tree like clusters
tree_mask = _label_tree_like_components(points_3d, ground_z,
                                       point_components,
                                       tree_points, min_height)
label_mask = tree_mask

### Noise filter on non tree points

In [ ]:
grid_size = 0.9
min_component_size = 50

lcc = LabelConnectedComp(grid_size=grid_size,
                         min_component_size=min_component_size)
point_components = lcc.get_components(points_3d[~label_mask])
cc_mask = point_components == -1
print(f'Found {np.count_nonzero(cc_mask)} noise points in '
             + f'clusters <{min_component_size} points.')

label_mask2 = np.zeros((len(points_3d),), dtype=bool)
# Label points below ground and points in small components.
label_mask2[~label_mask] = cc_mask

In [ ]:
labels = np.zeros((len(points_3d),), dtype='uint16')
labels[~label_mask] = 0
labels[label_mask] = 1
labels[label_mask2] = 99

In [ ]:
header = laspy.LasHeader(point_format=3, version="1.2")
header.offsets = las.header.offsets
header.scales = las.header.scales

new_las = laspy.LasData(header)

new_las.x = points_3d[:, 0]
new_las.y = points_3d[:, 1]
new_las.z = points_3d[:, 2]
new_las.red = points_rgb[:, 0]
new_las.green = points_rgb[:, 1]
new_las.blue = points_rgb[:, 2]

new_las.add_extra_dim(laspy.ExtraBytesParams(name="label", type="uint16",
                                         description="Label"))
new_las.label = labels

new_las.write(f'trees_{tilecode}.laz')